<a href="https://colab.research.google.com/github/MahiraPutri149/Islamophobic_Indonesia/blob/main/Twitter_Sentiment_Analysis_Support_Vector_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Basic Libraries

In [1]:
import sklearn
import numpy as np
import pandas as pd

# Import Data

In [2]:
df = pd.read_excel('dataaa.xlsx')

<ipython-input-2-e737fdf3306c>:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  df = pd.read_excel('dataaa.xlsx')


# Data Exploration (Exploratory Data Analysis)

In [3]:
df.head()

,Datetime,Text,Label
0,2022-05-27 06:04:01+00:00,Semoga yang benci islam atau sering giring opi...,0
1,2022-05-27 04:34:08+00:00,@Naollivia @MegaPKeliduan Tenggelamkan sampai ...,0
2,2022-05-27 04:24:52+00:00,@whyoedee @NUgarislucu Boleh benci dg Islam tp...,0
3,2022-05-27 00:50:14+00:00,Aku mendukung abang karena papua jg saudara k...,0
4,2022-05-26 21:46:38+00:00,"@dyahwie @frsp_ Buat ngehina lah, dia kan benc...",0


In [4]:
df.tail()

,Datetime,Text,Label
1577,2022-03-21 06:06:23+00:00,@Bambang_Budimab @Syarman59 @geloraco Kl benci...,0
1578,2022-03-21 05:29:57+00:00,@wardoyooioi @tifsembiring Sampeyan yg merasa ...,0
1579,2022-03-21 05:25:57+00:00,@katzedikke Autis amatiran bau kencur yaa bgit...,1
1580,2022-03-21 05:13:02+00:00,@LindaKa81442002 @Mdy_Asmara1701 Orang yang ng...,0
1581,2022-03-21 05:02:42+00:00,@jenny_jkt @yaniarsim Tp ama terorisme papua k...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   Datetime  1582 non-null   datetime64[ns, UTC]
 1   Text      1582 non-null   object             
 2   Label     1582 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(1)
memory usage: 37.2+ KB


In [6]:
# non-islamophobic related tweets
sum(df["Label"] == 0)

1194

In [7]:
# islamophobic related tweets
sum(df["Label"] == 1)

388

In [8]:
# check if there are any missing values
df.isnull().sum()

Datetime    0
Text        0
Label       0
dtype: int64

# Data cleaning

In [9]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [10]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [11]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [12]:
# clean training data
train_tweet = clean_tweets(df["Text"])
train_tweet = pd.DataFrame(train_tweet)

In [13]:
# append cleaned tweets to the training data
df["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
df.head(10)

,Datetime,Text,Label,clean_tweet
0,2022-05-27 06:04:01+00:00,Semoga yang benci islam atau sering giring opi...,0,semoga yang benci islam atau sering giring opi...
1,2022-05-27 04:34:08+00:00,@Naollivia @MegaPKeliduan Tenggelamkan sampai ...,0,tenggelamkan sampai dasar lautan manusia rasis...
2,2022-05-27 04:24:52+00:00,@whyoedee @NUgarislucu Boleh benci dg Islam tp...,0,boleh benci dg islam tp jgn melesetkan kt takb...
3,2022-05-27 00:50:14+00:00,Aku mendukung abang karena papua jg saudara k...,0,aku mendukung abang karena papua jg saudara ku...
4,2022-05-26 21:46:38+00:00,"@dyahwie @frsp_ Buat ngehina lah, dia kan benc...",0,buat ngehina lah dia kan benci islam
5,2022-05-26 21:46:00+00:00,Satu persatu akun2 kek @MegaPKeliduan yg rasis...,0,satu persatu akun2 kek yg rasis benci partai i...
6,2022-05-26 21:25:14+00:00,@Iqbalfas @f0rdd_ @FNazy14 @otterion @deadnerv...,1,kalo benci ya benci aja bang ga butuh alasan a...
7,2022-05-26 21:24:36+00:00,Gapernah sebelumnya tau doi ngapain aja sampe ...,0,gapernah sebelumnya tau doi ngapain aja sampe ...
8,2022-05-26 21:08:26+00:00,@rafajar8 @Adasaja92561423 @Hilmi28 @ustadabd...,1,kenapa benci somad karena dialah tipe bajingan...
9,2022-05-26 20:27:29+00:00,"@herumeneutic disatu sisi,lu ngajak orang untk...",0,disatu sisilu ngajak orang untk ikut mendukung...


In [14]:
df[df['Label']==0]

,Datetime,Text,Label,clean_tweet
0,2022-05-27 06:04:01+00:00,Semoga yang benci islam atau sering giring opi...,0,semoga yang benci islam atau sering giring opi...
1,2022-05-27 04:34:08+00:00,@Naollivia @MegaPKeliduan Tenggelamkan sampai ...,0,tenggelamkan sampai dasar lautan manusia rasis...
2,2022-05-27 04:24:52+00:00,@whyoedee @NUgarislucu Boleh benci dg Islam tp...,0,boleh benci dg islam tp jgn melesetkan kt takb...
3,2022-05-27 00:50:14+00:00,Aku mendukung abang karena papua jg saudara k...,0,aku mendukung abang karena papua jg saudara ku...
4,2022-05-26 21:46:38+00:00,"@dyahwie @frsp_ Buat ngehina lah, dia kan benc...",0,buat ngehina lah dia kan benci islam
...,...,...,...,...
1576,2022-03-21 06:12:47+00:00,@aliefxzn @Gustepra_96 Gausah ngomongin akidah...,0,gausah ngomongin akidah islam deh lu aja benci...
1577,2022-03-21 06:06:23+00:00,@Bambang_Budimab @Syarman59 @geloraco Kl benci...,0,kl benci islam terus terang ja blg bencong lo ...
1578,2022-03-21 05:29:57+00:00,@wardoyooioi @tifsembiring Sampeyan yg merasa ...,0,sampeyan yg merasa sok plural padahal nol besa...
1580,2022-03-21 05:13:02+00:00,@LindaKa81442002 @Mdy_Asmara1701 Orang yang ng...,0,orang yang ngaku islam seharusnya bangga denga...


In [15]:
df['clean_tweet'].to_excel('sentiment.xlsx')

In [ ]:
#Handling imbalance dataset
# from imblearn.over_sampling import SMOTE
# smote = SMOTE()
# X, y = smote.fit_resample(df['clean_tweet'], df['Label'])

# Test and Train split


In [16]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = df.Label.values

# use 80% for the training and 20% for the test
x_train, x_test, y_train, y_test = train_test_split(df.clean_tweet.values, y,
                                                    stratify=y,
                                                    random_state=1,
                                                    test_size=0.4, shuffle=True)

# Vectorize tweets using CountVectorizer

In [21]:
!git clone https://github.com/Braincore-id/IndoTWEES.git

Cloning into 'IndoTWEES'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 104 (delta 46), reused 45 (delta 12), pack-reused 0
Receiving objects: 100% (104/104), 114.16 KiB | 2.85 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [26]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

nltk_stop_words = set(stopwords.words('indonesian'))
len(nltk_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


757

In [27]:
data = 'https://raw.githubusercontent.com/Braincore-id/IndoTWEEST/main/stopwords_twitter.csv'

df_stopwords = pd.read_csv(data, names=['stopword'])
df_stopwords = df_stopwords.sort_values(by="stopword", ascending=True)
df_stopwords.reset_index(drop=True, inplace=True)

In [28]:
# Daftar stop words dalam bahasa Indonesia
indonesian_stop_words = ['dan', 'atau', 'adalah', 'saya','ya', 'iya', 'tak',
                         'aja', 'mau', 'jadi', 'jangan', 'pak', 'main', 'bukan',
                         'makin', 'nya', 'sih', 'orang']

In [29]:
# Gabungkan semua daftar stop words menjadi satu set
stop_words = set(indonesian_stop_words) | nltk_stop_words | set(df_stopwords['stopword'])

# Sekarang, all_stop_words berisi semua stop words dari berbagai sumber
print(len(stop_words))  # Jumlah total stop words

1039


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words=indonesian_stop_words)

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


# Model building

Apply Support Vetor Classifier (SVC)

In [31]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

# Accuracy score for SVC


In [32]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  69.826224328594 %


# Evaluation Report

In [33]:
from sklearn.metrics import confusion_matrix, classification_report
y_train_pred = svm.fit(x_train_vec, y_train).predict(x_train_vec)
y_test_pred = svm.fit(x_train_vec, y_train).predict(x_test_vec)

print('RANDOM FOREST TRAINING RESULT:')
print('======================================================')
print('Accuracy:', svm.fit(x_train_vec, y_train).score(x_train_vec, y_train))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_train, y_train_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_train, y_train_pred))

print('\n======================================================')

print('\n RANDOM FOREST TESTING RESULT:')
print('======================================================')
print('Accuracy:', svm.fit(x_train_vec, y_train).score(x_test_vec, y_test))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_test, y_test_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_test, y_test_pred))

RANDOM FOREST TRAINING RESULT:
Accuracy: 0.9978925184404637
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       716
           1       1.00      1.00      1.00       233

    accuracy                           1.00       949
   macro avg       1.00      1.00      1.00       949
weighted avg       1.00      1.00      1.00       949

Confussion Matrix ------------------------------------
[[715   1]
 [  1 232]]


 RANDOM FOREST TESTING RESULT:
Accuracy: 0.69826224328594
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       0.78      0.83      0.81       478
           1       0.35      0.28      0.31       155

    accuracy                           0.70       633
   macro avg       0.57      0.56      0.56       633
weighted avg       0.68      0.70      0.69       633

Confussion Matrix --------------------

#2nd Trial

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.id.stop_words import STOP_WORDS

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words=indonesian_stop_words)

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(df['clean_tweet'].values)

# transform documents to document-term matrix
x_vec = vectorizer.transform(df['clean_tweet'].values)

In [38]:
#Handling imbalance dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X, y = smote.fit_resample(x_vec, df['Label'])

In [39]:
from sklearn.model_selection import train_test_split

# use 80% for the training and 20% for the test
x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    random_state=1,
                                                    test_size=0.2, shuffle=True)

In [40]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(C=1, gamma=0.1, kernel='rbf', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train, y_train)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test)

In [41]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  79.07949790794979 %


In [42]:
from sklearn.metrics import confusion_matrix, classification_report
y_train_pred = svm.fit(x_train, y_train).predict(x_train)
y_test_pred = svm.fit(x_train, y_train).predict(x_test)

print('SUPPORT VECTOR MACHINE TRAINING RESULT:')
print('======================================================')
print('Accuracy:', svm.fit(x_train, y_train).score(x_train, y_train))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_train, y_train_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_train, y_train_pred))

print('\n======================================================')

print('\n SUPPORT VECTOR MACHINE TESTING RESULT:')
print('======================================================')
print('Accuracy:', svm.fit(x_train, y_train).score(x_test, y_test))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_test, y_test_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_test, y_test_pred))

SUPPORT VECTOR MACHINE TRAINING RESULT:
Accuracy: 0.9696335078534032
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       955
           1       0.96      0.98      0.97       955

    accuracy                           0.97      1910
   macro avg       0.97      0.97      0.97      1910
weighted avg       0.97      0.97      0.97      1910

Confussion Matrix ------------------------------------
[[913  42]
 [ 16 939]]


 SUPPORT VECTOR MACHINE TESTING RESULT:
Accuracy: 0.7907949790794979
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       0.75      0.88      0.81       239
           1       0.85      0.70      0.77       239

    accuracy                           0.79       478
   macro avg       0.80      0.79      0.79       478
weighted avg       0.80      0.79      0.79       478

Confussion Matrix 

#Hyper-Tuning

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC


tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                     'C': [0.5, 1],
                     'gamma': [0.1, 0.01],
                     'probability':[True, False]}
                   ]

clf = GridSearchCV(SVC(), tuned_parameters, scoring='accuracy')
clf.fit(x_test, y_test)


print("Best parameters set found on development set:\n")
print(clf.best_params_)
print("\nGrid scores on development set:\n")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

print("\nDetailed classification report:\n")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")

y_true, y_pred = y_test, clf.predict(x_test)

print(classification_report(y_true, y_pred))

Best parameters set found on development set:

{'C': 0.5, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}

Grid scores on development set:

0.667 (+/-0.106) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'linear', 'probability': True}
0.667 (+/-0.106) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'linear', 'probability': False}
0.508 (+/-0.040) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'poly', 'probability': True}
0.508 (+/-0.040) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'poly', 'probability': False}
0.782 (+/-0.125) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
0.782 (+/-0.125) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'rbf', 'probability': False}
0.653 (+/-0.136) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'sigmoid', 'probability': True}
0.653 (+/-0.136) for {'C': 0.5, 'gamma': 0.1, 'kernel': 'sigmoid', 'probability': False}
0.667 (+/-0.106) for {'C': 0.5, 'gamma': 0.01, 'kernel': 'linear', 'probability': True}
0.667 (+/-0.106) for {'C': 0.5, 'gamma': 0.01, 'kernel': 'linear', 'probabil

#Another model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(binary=True, stop_words=STOP_WORDS)
X_train_counts = count_vect.fit_transform(list(x_train) + list(x_test))
X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
X_train_tfidf

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
 ])

In [ ]:
text_clf.fit(X_train_tfidf, y_train)
predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
 }

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, cv=10, n_jobs=-1)

In [ ]:
gs_clf = gs_clf.fit(x_train, y_train)

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

#Using TFIDF

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.id.stop_words import STOP_WORDS

# create the transform
vectorizer = TfidfVectorizer(stop_words=indonesian_stop_words)

# tokenize and build vocab
vectorizer.fit(df['clean_tweet'].values)

x_vec = vectorizer.transform(df['clean_tweet'].values)

In [66]:
#Handling imbalance dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X, y = smote.fit_resample(x_vec, df['Label'])

In [67]:
from sklearn.model_selection import train_test_split

# use 80% for the training and 20% for the test
x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    random_state=1,
                                                    test_size=0.2)

In [68]:
from sklearn import svm
# classify using support vector classifier
svm_1 = svm.SVC(C=1, gamma=0.1, kernel='rbf', probability=True)

# fit the SVC model based on the given training data
prob = svm_1.fit(x_train, y_train)

# perform classification and prediction on samples in x_test
y_pred_svm = svm_1.predict(x_test)

In [69]:
from sklearn.metrics import confusion_matrix, classification_report
y_train_pred = svm_1.fit(x_train, y_train).predict(x_train)
y_test_pred = svm_1.fit(x_train, y_train).predict(x_test)

print('SUPPORT VECTOR MACHINE TRAINING RESULT:')
print('======================================================')
print('Accuracy:', svm_1.fit(x_train, y_train).score(x_train, y_train))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_train, y_train_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_train, y_train_pred))

print('\n======================================================')

print('\n SUPPORT VECTOR MACHINE TESTING RESULT:')
print('======================================================')
print('Accuracy:', svm_1.fit(x_train, y_train).score(x_test, y_test))
print('======================================================')
print('Classification Report --------------------------------')
print(classification_report(y_test, y_test_pred))
print('======================================================')
print('Confussion Matrix ------------------------------------')
print(confusion_matrix(y_test, y_test_pred))

SUPPORT VECTOR MACHINE TRAINING RESULT:
Accuracy: 0.8879581151832461
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       955
           1       0.95      0.82      0.88       955

    accuracy                           0.89      1910
   macro avg       0.90      0.89      0.89      1910
weighted avg       0.90      0.89      0.89      1910

Confussion Matrix ------------------------------------
[[913  42]
 [172 783]]


 SUPPORT VECTOR MACHINE TESTING RESULT:
Accuracy: 0.8535564853556485
Classification Report --------------------------------
              precision    recall  f1-score   support

           0       0.80      0.95      0.87       239
           1       0.93      0.76      0.84       239

    accuracy                           0.85       478
   macro avg       0.87      0.85      0.85       478
weighted avg       0.87      0.85      0.85       478

Confussion Matrix 

In [ ]:
scrappp['result'] = pd.DataFrame(prediction_result)

In [ ]:
scrappp[scrappp['result']==1]

In [ ]:
testify_vec2 = vectorizer.transform(scrappp['content'].values)

In [ ]:
prediction_result2 = svm_1.fit(x_train, y_train).predict(testify_vec2)

In [ ]:
scrappp['result_for_content'] = pd.DataFrame(prediction_result2)

In [ ]:
scrappp[scrappp['result_for_content']==1]

# Data Cleansing for Visualization

In [70]:
df['clean_tweet'].to_excel('cleaneddd.xlsx')

In [71]:
tempArr = []
for line in df['clean_tweet']:
  tempArr.append(line.split())

len(tempArr)

1582

In [72]:
df['split_words'] = tempArr

In [73]:
df.head()

,Datetime,Text,Label,clean_tweet,split_words
0,2022-05-27 06:04:01+00:00,Semoga yang benci islam atau sering giring opi...,0,semoga yang benci islam atau sering giring opi...,"[semoga, yang, benci, islam, atau, sering, gir..."
1,2022-05-27 04:34:08+00:00,@Naollivia @MegaPKeliduan Tenggelamkan sampai ...,0,tenggelamkan sampai dasar lautan manusia rasis...,"[tenggelamkan, sampai, dasar, lautan, manusia,..."
2,2022-05-27 04:24:52+00:00,@whyoedee @NUgarislucu Boleh benci dg Islam tp...,0,boleh benci dg islam tp jgn melesetkan kt takb...,"[boleh, benci, dg, islam, tp, jgn, melesetkan,..."
3,2022-05-27 00:50:14+00:00,Aku mendukung abang karena papua jg saudara k...,0,aku mendukung abang karena papua jg saudara ku...,"[aku, mendukung, abang, karena, papua, jg, sau..."
4,2022-05-26 21:46:38+00:00,"@dyahwie @frsp_ Buat ngehina lah, dia kan benc...",0,buat ngehina lah dia kan benci islam,"[buat, ngehina, lah, dia, kan, benci, islam]"


In [74]:
!pip install nltk

In [75]:
!python -m nltk.downloader popular

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_d

In [76]:
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.1 MB/s eta 0:00:00


In [77]:
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt

In [78]:
stop_words = set(stopwords.words('indonesian'))

word_tokens_no_stopwords = []

for i in df.split_words.values:
   for w in i:
      if not w in stop_words:
        word_tokens_no_stopwords.append(w)

In [79]:
textfile = open("clean_sentiment.txt", "w")
for element in word_tokens_no_stopwords:
    textfile.write(element + "\n")
textfile.close()